In [2]:
# import os
# import sys
# import gc
# import pickle
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torch.cuda.amp import autocast, GradScaler

# # ============================================================================
# # 1. CẤU HÌNH C99 (TUNED FOR TAIL)
# # ============================================================================
# CONFIG = {
#     'EMBED_DIR': "/kaggle/input/cafa6-embeds",
#     'LABEL_DIR': "/kaggle/input/c99-cafa6", # [QUAN TRỌNG] Đổi folder nếu cần
#     'WORK_DIR': "/kaggle/working",
    
#     # --- ĐỔI SANG FILE C99 ---
#     'VOCAB_FILE': "vocab_C99_remove.csv",
#     'TARGET_FILE': "train_targets_C99.pkl",
#     'TRAIN_IDS': "train_ids_C99_split.npy",
#     'VAL_IDS': "val_ids_C99_split.npy",
    
#     'IA_FILE': "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv",
    
#     # --- Model Params (Giữ nguyên) ---
#     'input_dim': 1280,
#     'hidden_dims': [2048, 4096], # Đủ mạnh cho C99
#     'dropout': 0.3,
#     # num_classes sẽ tự động load từ vocab
    
#     'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    
#     # --- Training Config ---
#     'batch_size': 16,    
#     'lr': 2e-4,          # Giữ nguyên tốc độ chuẩn
#     'weight_decay': 1e-4,
#     'epochs': 25         # 20 Epoch là đủ cho Standard
# }

# # ============================================================================
# # 2. MODEL & DATASET (GIỮ NGUYÊN BẢN GỐC)
# # ============================================================================
# class WideProteinMLP(nn.Module):
#     def __init__(self, input_dim, num_classes, hidden_dims=[2048, 4096], dropout=0.3):
#         super().__init__()
#         layers = [nn.LayerNorm(input_dim)]
#         prev = input_dim
#         for h in hidden_dims:
#             layers += [nn.Linear(prev, h), nn.GELU(), nn.Dropout(dropout)]
#             prev = h
#         layers.append(nn.Linear(prev, num_classes))
#         self.net = nn.Sequential(*layers)
#     def forward(self, x): return self.net(x)

# class CAFA6Dataset(Dataset):
#     def __init__(self, ids_file, targets_file, embed_dir, num_classes):
#         # Auto-detect path
#         path = os.path.join(CONFIG['LABEL_DIR'], ids_file)
#         if not os.path.exists(path): path = os.path.join(CONFIG['WORK_DIR'], ids_file)
#         self.ids = np.load(path)
        
#         t_path = os.path.join(CONFIG['LABEL_DIR'], targets_file)
#         if not os.path.exists(t_path): t_path = os.path.join(CONFIG['WORK_DIR'], targets_file)
#         with open(t_path, 'rb') as f: self.labels_dict = pickle.load(f)
            
#         self.num_classes = num_classes
#         self.id_to_embed_idx = {}
#         with open(os.path.join(embed_dir, "train_ids.txt"), 'r') as f:
#             for idx, line in enumerate(f): self.id_to_embed_idx[line.strip()] = idx
#         self.embed_matrix = np.load(os.path.join(embed_dir, "train_embeds.npy"), mmap_mode='r')

#     def __len__(self): return len(self.ids)
#     def __getitem__(self, idx):
#         prot_id = self.ids[idx]
#         embed_idx = self.id_to_embed_idx.get(prot_id)
#         feat = torch.from_numpy(self.embed_matrix[embed_idx].copy()).float() if embed_idx is not None else torch.zeros(1280)
#         target = torch.zeros(self.num_classes, dtype=torch.float)
#         indices = self.labels_dict.get(prot_id, [])
#         if len(indices) > 0: target[indices] = 1.0
#         return feat, target

# # ============================================================================
# # 3. LOSS FUNCTION (ASL)
# # ============================================================================
# class AsymmetricLossOptimized(nn.Module):
#     def __init__(self, gamma_neg=4, gamma_pos=0, clip=0.05, eps=1e-8):
#         super().__init__()
#         self.gamma_neg = gamma_neg
#         self.gamma_pos = gamma_pos
#         self.clip = clip
#         self.eps = eps

#     def forward(self, x, y):
#         x_sigmoid = torch.sigmoid(x)
#         xs_pos = x_sigmoid
#         xs_neg = 1 - x_sigmoid
#         if self.clip > 0: xs_neg = (xs_neg + self.clip).clamp(max=1)
        
#         pt = y * xs_pos + (1 - y) * xs_neg
#         log_pt = torch.log(pt.clamp(min=self.eps))
        
#         pos_weight = (1 - xs_pos) ** self.gamma_pos
#         neg_weight = (1 - xs_neg) ** self.gamma_neg
        
#         loss = - (pos_weight * log_pt * y + neg_weight * log_pt * (1-y))
#         return loss.sum()

# # ============================================================================
# # 4. METRIC & MAIN (DÙNG HÀM METRIC CHUẨN CỦA BẠN)
# # ============================================================================
# def calculate_fmax_subset(preds, targets, ia_weights):
#     w = ia_weights.reshape(1, -1)
#     true_sum = np.sum(targets * w, axis=1)
#     valid_mask = true_sum > 0
#     if valid_mask.sum() == 0: return 0.0
    
#     p_sub = preds[valid_mask]
#     t_sub = targets[valid_mask]
#     w_sub = w 
#     w_true_sub = true_sum[valid_mask]
    
#     best_f1 = 0.0
#     # Quét 51 ngưỡng cho nhanh
#     thresholds = np.linspace(0.0, 1.0, 51) 
    
#     for tau in thresholds:
#         cut = (p_sub >= tau).astype(int)
#         tp = np.sum((cut * t_sub) * w_sub, axis=1)
#         pred_sum = np.sum(cut * w_sub, axis=1)
        
#         prec = np.divide(tp, pred_sum, out=np.zeros_like(tp), where=pred_sum!=0)
#         rec = np.divide(tp, w_true_sub, out=np.zeros_like(tp), where=w_true_sub!=0)
        
#         avg_p = np.mean(prec)
#         avg_r = np.mean(rec)
        
#         if (avg_p + avg_r) > 0:
#             f1 = 2 * avg_p * avg_r / (avg_p + avg_r)
#         else:
#             f1 = 0.0
            
#         if f1 > best_f1: best_f1 = f1
            
#     return best_f1

# def validate_detailed(model, loader, vocab_df, ia_weights, device):
#     model.eval()
#     all_preds, all_targets = [], []
#     with torch.no_grad():
#         for x, y in loader:
#             x = x.to(device)
#             with autocast():
#                 logits = model(x)
#             all_preds.append(torch.sigmoid(logits).cpu().numpy())
#             all_targets.append(y.numpy())
#     Y_p = np.vstack(all_preds)
#     Y_t = np.vstack(all_targets)
    
#     scores = {}
#     # Lưu ý: C99 có nhiều nhãn hơn, nên việc loop aspect vẫn đúng
#     for aspect in ['MFO', 'BPO', 'CCO']:
#         col_indices = vocab_df.index[vocab_df['aspect'] == aspect].tolist()
#         if not col_indices: continue
#         scores[aspect] = calculate_fmax_subset(Y_p[:, col_indices], Y_t[:, col_indices], ia_weights[col_indices])
    
#     avg_fmax = np.mean(list(scores.values()))
#     return avg_fmax, scores

# def main():
#     print(f"🚀 START TRAINING C99 STANDARD (WIDE MLP)...")
    
#     # 1. Load Vocab C99
#     path = os.path.join(CONFIG['LABEL_DIR'], CONFIG['VOCAB_FILE'])
#     if not os.path.exists(path): path = os.path.join(CONFIG['WORK_DIR'], CONFIG['VOCAB_FILE'])
#     vocab_df = pd.read_csv(path)
#     num_classes = len(vocab_df)
#     print(f"   Num Classes: {num_classes}")
    
#     # Load IA Weights (Map theo Vocab C99)
#     try:
#         ia_df = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'], header=None)
#         ia_map = dict(zip(ia_df.term, ia_df.ia))
#         ia_weights = np.array([ia_map.get(t, 1.0) for t in vocab_df.term.values])
#     except: ia_weights = np.ones(num_classes)
    
#     # 2. Loaders
#     train_ds = CAFA6Dataset(CONFIG['TRAIN_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], num_classes)
#     val_ds = CAFA6Dataset(CONFIG['VAL_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], num_classes)
#     train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
#     val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size']*2, shuffle=False, num_workers=2)
    
#     # 3. Model
#     model = WideProteinMLP(CONFIG['input_dim'], num_classes, CONFIG['hidden_dims'], CONFIG['dropout']).to(CONFIG['device'])
#     if torch.cuda.device_count() > 1: model = nn.DataParallel(model)
    
#     # 4. Config Tối ưu cho C99 (GammaPos=1.0)
#     # [QUAN TRỌNG]: Tăng gamma_pos lên 1.0 để bắt nhãn hiếm (C99 nhiều nhãn hiếm)
#     criterion = AsymmetricLossOptimized(gamma_neg=2.0, gamma_pos=2.0, clip=0.05)
    
#     optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=CONFIG['lr'], steps_per_epoch=len(train_loader), epochs=CONFIG['epochs'])
#     scaler = GradScaler()

#     best_score = 0.0
    
#     # 5. Training Loop
#     for epoch in range(CONFIG['epochs']):
#         model.train()
#         loss_sum = 0
#         pbar = tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False)
        
#         for x, y in pbar:
#             x, y = x.to(CONFIG['device']), y.to(CONFIG['device'])
#             optimizer.zero_grad()
#             with autocast():::
#                 logits = model(x)
#                 loss = criterion(logits, y)
            
#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()
#             scheduler.step()
#             loss_sum += loss.item()
#             pbar.set_postfix({'loss': f"{loss.item():.4f}"})
            
#         # Validate
#         val_fmax, val_details = validate_detailed(model, val_loader, vocab_df, ia_weights, CONFIG['device'])
#         print(f"Epoch {epoch+1}: Loss={loss_sum/len(train_loader):.4f} | Val F-max={val_fmax:.4f} {val_details}")
        
#         if val_fmax > best_score:
#             best_score = val_fmax
#             torch.save(model.state_dict(), "best_model_c99_extreme.pth")
#             print("   🏆 Saved Best C99 Model!")

#     print(f"\n✅ DONE! Best F-max: {best_score:.4f}")

# if __name__ == "__main__":
#     main()

In [4]:
# !pip install obonet networkx --quiet

# import obonet
# import networkx as nx
# import pandas as pd
# import numpy as np
# import pickle
# from collections import deque
# import os

# CONFIG = {
#     'OBO_FILE': "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo",
#     'VOCAB_FILE': "/kaggle/input/c99-cafa6/vocab_C99_remove.csv", 
#     'TRAIN_TARGETS': "/kaggle/input/c99-cafa6/train_targets_C99.pkl", # Cần cho tính IC
#     'OUTPUT_PKL': "hierarchy_metadata_C99.pkl"
# }

# def build_training_metadata_full():
#     print("🚀 Building Training Metadata (Depth, IC & HMC Map)...")
    
#     # 1. Load Graph & Vocab C95
#     graph = obonet.read_obo(CONFIG['OBO_FILE'])
#     vocab_df = pd.read_csv(CONFIG['VOCAB_FILE'])
#     vocab_terms = vocab_df['term'].tolist()
#     go_to_idx = {go: i for i, go in enumerate(vocab_terms)}
#     n_classes = len(vocab_terms)
    
#     # ----------------------------------------------------
#     # 2. TÍNH DEPTH (BFS) - Logic của bạn
#     # ----------------------------------------------------
#     roots = ["GO:0008150", "GO:0003674", "GO:0005575"]
#     depth_dict = {node: float('inf') for node in graph.nodes()}
#     queue = deque()
    
#     for root in roots:
#         if root in graph:
#             depth_dict[root] = 0
#             queue.append(root)
            
#     while queue:
#         node = queue.popleft()
#         d = depth_dict[node]
#         # Sử dụng 'is_a' và 'part_of' edges (chỉ cần lấy edge predecessors)
#         for child in graph.predecessors(node):
#             if depth_dict[child] > d + 1:
#                 depth_dict[child] = d + 1
#                 queue.append(child)
                
#     # Map Depth vào Vocab C95
#     depth_arr = np.array([depth_dict.get(t, 0) for t in vocab_terms], dtype=np.float32)
#     depth_arr[depth_arr == float('inf')] = 0.0
    
#     # Normalize
#     max_d = depth_arr.max()
#     depth_norm = depth_arr / max_d if max_d > 0 else depth_arr
#     print(f"  ✅ Depth Done. Max Depth: {max_d}")
    
#     # ----------------------------------------------------
#     # 3. TÍNH INFORMATION CONTENT (IC)
#     # ----------------------------------------------------
#     # Load targets để tính tần suất
#     with open(CONFIG['TRAIN_TARGETS'], 'rb') as f:
#         labels_dict = pickle.load(f)
    
#     # Tính tần suất (Count)
#     term_counts = np.zeros(n_classes, dtype=np.int32)
#     total_proteins = len(labels_dict)

#     # Tổng hợp nhãn từ dictionary
#     for pid, indices in labels_dict.items():
#         # Chỉ cần đảm bảo index không vượt quá n_classes (đã được lọc C95)
#         for idx in indices:
#             if idx < n_classes:
#                 term_counts[idx] += 1
    
#     # Công thức IC: IC = -log(P(t))
#     term_prob = term_counts / total_proteins
#     # Thêm epsilon nhỏ để tránh log(0)
#     term_prob = np.clip(term_prob, 1e-8, 1.0)
#     ic_arr = -np.log(term_prob)
    
#     # Normalize IC (Optional, nhưng an toàn hơn)
#     max_ic = ic_arr.max()
#     ic_norm = ic_arr / max_ic if max_ic > 0 else ic_arr
#     print(f"  ✅ IC Done. Max IC: {max_ic:.2f}")

#     # ----------------------------------------------------
#     # 4. CHILD-TO-PARENT INDEX MAP (Cho HMC Loss)
#     # ----------------------------------------------------
#     child_to_parent_idx = {}
    
#     # Chỉ duyệt qua các GO Term CÓ trong vocab C95
#     for term_go in tqdm(vocab_terms, desc="Building Parent Map"):
#         if term_go not in graph: continue
        
#         child_idx = go_to_idx[term_go]
#         parent_indices = []
        
#         # predecessors = parents trong graph
#         for parent_go in graph.successors(term_go):
#             if parent_go in go_to_idx:
#                 parent_indices.append(go_to_idx[parent_go])
        
#         if parent_indices:
#             child_to_parent_idx[child_idx] = parent_indices

#     print(f"  ✅ HMC Map Done. Total nodes mapped: {len(child_to_parent_idx):,}")
    
#     # 5. Save
#     metadata = {
#         'depth_norm': depth_norm,
#         'ic_norm': ic_norm,
#         'child_to_parent_idx': child_to_parent_idx
#     }
#     with open(CONFIG['OUTPUT_PKL'], 'wb') as f:
#         pickle.dump(metadata, f)
        
#     print(f"\n✅ DONE! Saved all metadata to {CONFIG['OUTPUT_PKL']}")

# if __name__ == "__main__":
#     # Đảm bảo file TRAIN_TARGETS đã được mount hoặc tạo ra
#     if not os.path.exists(CONFIG['TRAIN_TARGETS']):
#         print(f"🚨 ERROR: Missing {CONFIG['TRAIN_TARGETS']}. Cannot calculate IC.")
#     else:
#         build_training_metadata_full()

In [ ]:
import os, sys, math, time, pickle, gc

import numpy as np

import pandas as pd

from tqdm import tqdm

import torch

import torch.nn as nn

import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

from torch.amp import autocast, GradScaler


# ============================================================================

# 1. CONFIG

# ============================================================================

CONFIG = {

    # --- Input Embeddings ---
    "EMBED_DIR": "/kaggle/input/cafa6-embeds",

    # --- Input Labels & Metadata (Dataset c95-cafa6) ---
    "WORK_DIR": "/kaggle/working",
    
    "LABEL_DIR": "/kaggle/input/c99-cafa6",

    'VOCAB_FILE': "vocab_C99_remove.csv",
    'TARGET_FILE': "train_targets_C99.pkl",
    'TRAIN_IDS': "train_ids_C99_split.npy",
    'VAL_IDS': "val_ids_C99_split.npy",
    
    "IA_FILE": "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv",

    'TAXON_PKL': "/kaggle/input/cafa6-embeds/taxon_mapping_K140.pkl",

    "PRETRAINED_PATH": "/kaggle/input/model-cafa6/best_model_c99_taxon.pth",

    # --- Model Params ---
    "input_dim": 1280,
    "hidden_dims": [2048, 4096],
    "dropout": 0.3,
    'taxon_embed_dim': 64,
    
    "batch_size": 16,
    "lr": 5e-5,
    "weight_decay": 1e-5,
    "epochs": 10,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
}

# ============================================================================
# 2. MODEL MỚI (CÓ TAXONOMY)
# ============================================================================

class WideProteinMLP_WithTaxon(nn.Module):
    def __init__(self, input_dim, num_classes, num_taxa, taxon_dim=64, hidden_dims=[2048, 4096], dropout=0.3):
        super().__init__()
        
        # 1. Nhánh Protein (ESM-2) - Chuẩn hóa đầu vào
        self.bn_input = nn.LayerNorm(input_dim)
        
        # 2. Nhánh Taxonomy - Embedding học được
        self.taxon_embedding = nn.Embedding(num_taxa, taxon_dim)
        self.taxon_norm = nn.LayerNorm(taxon_dim)  

        self.unk_idx = num_taxa - 1

        with torch.no_grad():
            self.taxon_embedding.weight[self.unk_idx].zero_()
        
        self.register_buffer(
            "unk_fixed_vector",
            self.taxon_embedding.weight[self.unk_idx].clone()
        )
        
        # 3. Tổng hợp (Concat)
        combined_dim = input_dim + taxon_dim
        
        layers = []
        prev = combined_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
            prev = h
            
        layers.append(nn.Linear(prev, num_classes))
        self.net = nn.Sequential(*layers)
        
    def forward(self, x_seq, x_tax):
        # x_seq: [batch, 1280]
        # x_tax: [batch] (Int IDs)
        
        feat_seq = self.bn_input(x_seq)
        feat_tax = self.taxon_embedding(x_tax)
        feat_tax = self.taxon_norm(feat_tax)  
        
        # Nối lại: [Batch, 1280 + 64]
        combined = torch.cat([feat_seq, feat_tax], dim=1)
        
        return self.net(combined)

# ============================================================================
# 3. DATASET CẬP NHẬT (LOAD TAXON)
# ============================================================================
class CAFA6Dataset(Dataset):
    def __init__(self, ids_file, targets_file, embed_dir, num_classes, taxon_pkl):
        # Load ID & Labels như cũ
        path = os.path.join(CONFIG['LABEL_DIR'], ids_file)
        if not os.path.exists(path): path = os.path.join(CONFIG['WORK_DIR'], ids_file)
        self.ids = np.load(path)
        
        t_path = os.path.join(CONFIG['LABEL_DIR'], targets_file)
        if not os.path.exists(t_path): t_path = os.path.join(CONFIG['WORK_DIR'], targets_file)
        with open(t_path, 'rb') as f: self.labels_dict = pickle.load(f)
            
        self.num_classes = num_classes
        self.id_to_embed_idx = {}
        with open(os.path.join(embed_dir, "train_ids.txt"), 'r') as f:
            for idx, line in enumerate(f): self.id_to_embed_idx[line.strip()] = idx
        self.embed_matrix = np.load(os.path.join(embed_dir, "train_embeds.npy"), mmap_mode='r')
        
        # [MỚI] Load Taxon Mapping
        # Tìm file ở work dir hoặc input dir
        
        if os.path.exists(taxon_pkl):
            tax_path = taxon_pkl
        else:
            tax_path = os.path.join(CONFIG['WORK_DIR'], os.path.basename(taxon_pkl))
            print("⚠️ USING TAXON PKL FROM WORK_DIR:", tax_path)
            
        with open(tax_path, 'rb') as f:
            tax_data = pickle.load(f)
        
        self.prot_to_taxon = tax_data['prot_to_taxon_idx'] # Key trong file pkl của bạn
        self.default_tax = tax_data['num_taxa_classes'] - 1 # Lớp UNK (Index cuối cùng)

    def __len__(self): return len(self.ids)

    def __getitem__(self, idx):
        prot_id = self.ids[idx]
        
        # 1. Embed
        embed_idx = self.id_to_embed_idx.get(prot_id)
        if embed_idx is None:
            feat = torch.zeros(CONFIG["input_dim"], dtype=torch.float32)
        else:
            feat = torch.from_numpy(self.embed_matrix[embed_idx].copy()).float()
        
        # 2. Target
        target = torch.zeros(self.num_classes, dtype=torch.float)
        indices = self.labels_dict.get(prot_id, [])
        if len(indices) > 0: target[indices] = 1.0
            
        # 3. [MỚI] Taxon ID
        # Lấy Taxon Index, nếu không có thì trả về default (UNK)
        taxon_idx = self.prot_to_taxon.get(prot_id, self.default_tax)
        
        # Trả về 3 giá trị
        return feat, torch.tensor(taxon_idx, dtype=torch.long), target


# ============================================================================

# 4. LOSS: ASL OPTIMIZED 

# ============================================================================


class AsymmetricLossOptimized(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=0, clip=0.05, eps=1e-8):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps
    def forward(self, x, y):
        x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid
        if self.clip > 0: xs_neg = (xs_neg + self.clip).clamp(max=1)
        pt = y * xs_pos + (1 - y) * xs_neg
        log_pt = torch.log(pt.clamp(min=self.eps))
        pos_weight = (1 - xs_pos) ** self.gamma_pos
        neg_weight = (1 - xs_neg) ** self.gamma_neg
        weighted_loss = - (pos_weight * log_pt * y + neg_weight * log_pt * (1-y))
        return weighted_loss.sum() / x.size(0)

# ============================================================================

# 5. METRIC & TRAINING LOOP

# ============================================================================


def calculate_fmax_subset(preds, targets, ia_weights):
    w = ia_weights.reshape(1, -1)
    true_sum = np.sum(targets * w, axis=1)
    valid_mask = true_sum > 0
    if valid_mask.sum() == 0: return 0.0
    p_sub = preds[valid_mask]; t_sub = targets[valid_mask]; w_sub = w; w_true_sub = true_sum[valid_mask]
    best_f1 = 0.0
    thresholds = np.linspace(0.0, 1.0, 51) 
    for tau in thresholds:
        cut = (p_sub >= tau).astype(int)
        tp = np.sum((cut * t_sub) * w_sub, axis=1)
        pred_sum = np.sum(cut * w_sub, axis=1)
        prec = np.divide(tp, pred_sum, out=np.zeros_like(tp), where=pred_sum!=0)
        rec = np.divide(tp, w_true_sub, out=np.zeros_like(tp), where=w_true_sub!=0)
        avg_p = np.mean(prec); avg_r = np.mean(rec)
        if (avg_p + avg_r) > 0: f1 = 2 * avg_p * avg_r / (avg_p + avg_r)
        else: f1 = 0.0
        if f1 > best_f1: best_f1 = f1
    return best_f1

def validate_with_loss(model, loader, vocab_df, ia_weights, device, criterion):
    model.eval()
    all_preds, all_targets = [], []
    val_loss_sum = 0.0
    n_batches = 0

    with torch.no_grad():
        for x_seq, x_tax, y in loader:
            x_seq = x_seq.to(device)
            x_tax = x_tax.to(device)
            y = y.to(device)

            with autocast(device_type="cuda"):
                logits = model(x_seq, x_tax)
                loss = criterion(logits, y)

            val_loss_sum += loss.item()
            n_batches += 1

            all_preds.append(torch.sigmoid(logits).cpu().numpy())
            all_targets.append(y.cpu().numpy())

    val_loss = val_loss_sum / n_batches

    Y_p = np.vstack(all_preds)
    Y_t = np.vstack(all_targets)

    scores = {}
    for aspect in ['MFO', 'BPO', 'CCO']:
        col_indices = vocab_df.index[vocab_df['aspect'] == aspect].tolist()
        if not col_indices:
            continue
        scores[aspect] = calculate_fmax_subset(
            Y_p[:, col_indices], 
            Y_t[:, col_indices], 
            ia_weights[col_indices]
        )

    avg_fmax = np.mean(list(scores.values()))
    return val_loss, avg_fmax, scores


# ============================================================================
# 5. RESUME TRAINING LOOP (Đã sửa lại)
# ============================================================================
def resume_c99_taxon():
    print("🚀 RESUMING C99 TAXON (FINE-TUNE 10 EPOCHS)...")
    
    # 1. Load Resources
    vocab_df = pd.read_csv(os.path.join(CONFIG['LABEL_DIR'], CONFIG['VOCAB_FILE']))
    num_classes = len(vocab_df)
    try:
        ia_df = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'], header=None)
        ia_map = dict(zip(ia_df.term, ia_df.ia))
        ia_weights = np.array([ia_map.get(t, 1.0) for t in vocab_df.term.values])
    except: ia_weights = np.ones(num_classes)
    
    tax_pkl_path = CONFIG['TAXON_PKL'] if os.path.exists(CONFIG['TAXON_PKL']) else os.path.join(CONFIG['WORK_DIR'], os.path.basename(CONFIG['TAXON_PKL']))
    with open(tax_pkl_path, 'rb') as f: tax_data = pickle.load(f)
    num_taxa = tax_data['num_taxa_classes']
    print(f"   Num Taxa Classes: {num_taxa}")

    # 2. Dataset & Loader
    train_ds = CAFA6Dataset(CONFIG['TRAIN_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], num_classes, CONFIG['TAXON_PKL'])
    val_ds = CAFA6Dataset(CONFIG['VAL_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], num_classes, CONFIG['TAXON_PKL'])
    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size']*2, shuffle=False, num_workers=2)
    
    # 3. Init Model & Load Weights
    model = WideProteinMLP_WithTaxon(
        input_dim=CONFIG['input_dim'], 
        num_classes=num_classes, 
        num_taxa=num_taxa,
        taxon_dim=CONFIG['taxon_embed_dim'],
        hidden_dims=CONFIG['hidden_dims'], 
        dropout=CONFIG['dropout']
    ).to(CONFIG['device'])
    
    # --- LOAD WEIGHTS TỪ EPOCH 15 ---
    print(f"📥 Loading weights from: {CONFIG['PRETRAINED_PATH']}")
    if os.path.exists(CONFIG['PRETRAINED_PATH']):
        state_dict = torch.load(CONFIG['PRETRAINED_PATH'], map_location=CONFIG['device'])
        # Fix lỗi module nếu train bằng DataParallel trước đó
        new_state_dict = {}
        for k, v in state_dict.items():
            if k.startswith("module."): new_state_dict[k[7:]] = v
            else: new_state_dict[k] = v
        model.load_state_dict(new_state_dict)
        print("✅ Weights loaded successfully!")
    else:
        print(f"❌ ERROR: Không tìm thấy file {CONFIG['PRETRAINED_PATH']}!")
        return

    if torch.cuda.device_count() > 1: model = nn.DataParallel(model)
    
    # 4. Setup Optimizer Mới (LR nhỏ + Cosine Decay)
    criterion = AsymmetricLossOptimized(gamma_neg=3.0, gamma_pos=1.2, clip=0.05)
    
    # Reset Optimizer với LR nhỏ
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
    
    # Dùng CosineAnnealing để hạ cánh mềm
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['epochs'], eta_min=1e-6)
    scaler = GradScaler("cuda") if torch.cuda.is_available() else GradScaler()

    best_score = 0.5637 # Điểm epoch 15 cũ của bạn
    
    # 5. Training Loop
    for epoch in range(CONFIG['epochs']):
        real_epoch = 15 + epoch + 1 # Để log hiển thị đúng epoch 16, 17...
        
        model.train()
        loss_sum = 0
        pbar = tqdm(train_loader, desc=f"Ep {real_epoch}", leave=False)
        
        for x_seq, x_tax, y in pbar:
            x_seq = x_seq.to(CONFIG['device'])
            x_tax = x_tax.to(CONFIG['device'])
            y = y.to(CONFIG['device'])
            
            optimizer.zero_grad()
            with autocast(device_type="cuda"):
                logits = model(x_seq, x_tax)
                loss = criterion(logits, y)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # ✅ LOGIC GIỮ NGUYÊN: CHẶN UNK HỌC LẠI
            with torch.no_grad():
                if isinstance(model, nn.DataParallel):
                    emb = model.module.taxon_embedding
                    unk_idx = model.module.unk_idx
                    unk_vec = model.module.unk_fixed_vector
                else:
                    emb = model.taxon_embedding
                    unk_idx = model.unk_idx
                    unk_vec = model.unk_fixed_vector
                emb.weight[unk_idx].copy_(unk_vec)
            
            # Scheduler Step theo Epoch (với CosineAnnealing), KHÔNG step trong batch
            
            loss_sum += loss.item()
            pbar.set_postfix({'loss': f"{loss.item():.4f}"})

        # Step scheduler cuối epoch
        scheduler.step()
        
        # Validate mỗi epoch (vì đang giai đoạn cuối quan trọng)
        val_loss, val_fmax, val_details = validate_with_loss(
            model, val_loader, vocab_df, ia_weights, CONFIG['device'], criterion
        )
        
        print(
            f"Epoch {real_epoch}: "
            f"Train Loss={loss_sum/len(train_loader):.4f} | "
            f"Val Loss={val_loss:.4f} | "
            f"Val F-max={val_fmax:.4f} {val_details}"
        )
        
        if val_fmax > best_score:
            best_score = val_fmax
            torch.save(model.state_dict(), "best_model_c99_taxon_final.pth")
            print("   🏆 Saved New Best Model!")
        
        # Luôn save checkpoint cuối cùng
        torch.save(model.state_dict(), "last_checkpoint.pth")

if __name__ == "__main__":
    resume_c99_taxon()


🚀 RESUMING C99 TAXON (FINE-TUNE 10 EPOCHS)...
   Num Taxa Classes: 141
📥 Loading weights from: /kaggle/input/model-cafa6/best_model_c99_taxon.pth
✅ Weights loaded successfully!


Epoch 16: Train Loss=20.3428 | Val Loss=36.2146 | Val F-max=0.5757 {'MFO': 0.6444812960287448, 'BPO': 0.4576959253703782, 'CCO': 0.6247965562551481}
   🏆 Saved New Best Model!


Epoch 17: Train Loss=19.7739 | Val Loss=36.2949 | Val F-max=0.5762 {'MFO': 0.6451744691688489, 'BPO': 0.45844852996905644, 'CCO': 0.6250407320894099}
   🏆 Saved New Best Model!


Epoch 18: Train Loss=19.1909 | Val Loss=31.5091 | Val F-max=0.5772 {'MFO': 0.6459181905668602, 'BPO': 0.4605007760652698, 'CCO': 0.6250537249918373}
   🏆 Saved New Best Model!


Epoch 19: Train Loss=18.3193 | Val Loss=31.8570 | Val F-max=0.5777 {'MFO': 0.64491136375324, 'BPO': 0.4618140515001747, 'CCO': 0.6262926421429607}
   🏆 Saved New Best Model!


...

Epoch 21: Train Loss=16.9935 | Val Loss=32.1445 | Val F-max=0.5783 {'MFO': 0.6461223303097027, 'BPO': 0.46286250282369623, 'CCO': 0.6258642920640908}
   🏆 Saved New Best Model!


Epoch 22: Train Loss=16.7039 | Val Loss=32.3557 | Val F-max=0.5790 {'MFO': 0.6467151929842894, 'BPO': 0.46332864905447296, 'CCO': 0.6268752261281997}
   🏆 Saved New Best Model!


Epoch 23: Train Loss=16.4604 | Val Loss=32.4664 | Val F-max=0.5795 {'MFO': 0.6472463836202468, 'BPO': 0.463818548497588, 'CCO': 0.6275731062879047}
   🏆 Saved New Best Model!


Epoch 24: Train Loss=16.3439 | Val Loss=32.5050 | Val F-max=0.5792 {'MFO': 0.6468590678372205, 'BPO': 0.46410387635460354, 'CCO': 0.6267588582664426}


Ep 25:  12%|█▏        | 551/4636 [00:53<06:37, 10.28it/s, loss=20.1166]